In [1]:
!pip install transformers datasets peft
!pip install torch --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_path = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = 'right'

fine_tuned_model_path = "/content/drive/MyDrive/Colab Notebooks/phi3-finetuned"

model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(model, fine_tuned_model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

# Load opus-100 dataset
dataset = load_dataset("Helsinki-NLP/opus-100", "en-id")

# Load test data
test_data = dataset["test"]

test_data[0]

README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/132k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': 'He was on the run.',
  'id': 'Dia sedang dalam pelarian.'}}

In [5]:
def generate_few_shot_prompt(english_sentences, indonesian_targets, new_sentence):
    """
    Generate a few-shot prompt for English to Indonesian translation.

    Parameters:
        english_sentences (list): List of example English sentences.
        indonesian_targets (list): List of corresponding Indonesian translations.
        new_sentence (str): The new English sentence to translate.

    Returns:
        str: Formatted few-shot prompt.
    """
    # Validate inputs
    if len(english_sentences) != len(indonesian_targets):
        raise ValueError("The number of English sentences and Indonesian targets must match.")

    # Start with "Examples"
    examples = ""
    for eng, indo in zip(english_sentences, indonesian_targets):
        examples += f"English: {eng}\nIndonesian: {indo}\n\n"

    # Add the new sentence for translation
    prompt = (
        examples +
        "translate English to Indonesian: \n"
        + f"\nEnglish: {new_sentence}\nIndonesian:"
    )

    return prompt

In [6]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure the nltk 'punkt' tokenizer is downloaded
nltk.download('punkt_tab')

def calculate_bleu_single_reference(reference_sentence, candidate_sentence):
    """
    Calculate the BLEU score for a single predicted translation against one reference.

    Parameters:
        reference_sentence (str): The correct reference translation.
        candidate_sentence (str): The predicted translation.

    Returns:
        float: The BLEU score.
    """
    # Tokenize the reference sentence and candidate sentence
    tokenized_reference = [nltk.word_tokenize(reference_sentence)]
    tokenized_candidate = nltk.word_tokenize(candidate_sentence)

    # Use smoothing to handle short sentences
    smoothing_function = SmoothingFunction().method1

    # Calculate BLEU score
    bleu_score = sentence_bleu(tokenized_reference, tokenized_candidate, smoothing_function=smoothing_function)
    return bleu_score

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
# Test Data Examples
# english_sentences =
#     "Can I help you?"
#     "As you can see, I'm jet-lagged and trying to adjust to the time-zone change."
#     "That's what the policeman just said."
#     "Yeah, we've been teaching him, but really, he's been teaching us."
#     "Someone wants to kill him."
#     "Hey Four, that's pretty good right?"
#     "You should've told me, boy."
#     "Your bed is in the corner, over there."
#     "Hey, kid, you've done really good so far, so what we're gonna do right now is we're gonna call your daddy."

# indonesian_targets =
#     "ada yang bisa kubantu?"
#     "Seperti yang kamu lihat , aku alami jet - lag dan mencoba untuk menyesuaikan diri dengan perubahan zona waktu ."
#     "Begitulah kata polisinya tadi."
#     "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
#     "Seseorang ingin membunuhnya."
#     "Four, Tak begitu buruk, kan?"
#     "Kau harusnya katakan padaku, Nak."
#     "Ranjangmu disudut itu."
#     "Hei, nak, sejauh ini kau telah melakukannya dengan baik sekali, jadi apa yang akan kita lakukan sekarang adalah kita akan menelpon ayahmu."

# Training Data Examples
english_sentences = [
    "I feel sorry for her.",
    "I am always with you.",
    "We didn't understand much of what he said, only that he was suffering.",
    "Hey, good morning!",
    "That's never happened.",
    "It's just, the way I asked you, I kind of left out the most important part."
]

indonesian_targets = [
    "Aku merasa kasihan padanya.",
    "Saya akan selalu bersamamu.",
    "Kami tidak mengerti apa yang ia katakan, selain bahwa dia menderita.",
    "Hei, selamat pagi!",
    "Itu tidak pernah terjadi.",
    "Hanya saja, caraku menanyakannya padamu, aku agak merasa kelewatan bagian yang paling pentingnya."
]

# The new sentence to translate
new_sentence = "Yeah, we've been teaching him, but really, he's been teaching us."

# Generate the prompt
prompt = generate_few_shot_prompt(english_sentences, indonesian_targets, new_sentence)
print(prompt)

English: I feel sorry for her.
Indonesian: Aku merasa kasihan padanya.

English: I am always with you.
Indonesian: Saya akan selalu bersamamu.

English: We didn't understand much of what he said, only that he was suffering.
Indonesian: Kami tidak mengerti apa yang ia katakan, selain bahwa dia menderita.

English: Hey, good morning!
Indonesian: Hei, selamat pagi!

English: That's never happened.
Indonesian: Itu tidak pernah terjadi.

English: It's just, the way I asked you, I kind of left out the most important part.
Indonesian: Hanya saja, caraku menanyakannya padamu, aku agak merasa kelewatan bagian yang paling pentingnya.

translate English to Indonesian: 

English: Yeah, we've been teaching him, but really, he's been teaching us.
Indonesian:


In [8]:
# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)

output = model.generate(
      input_ids=inputs,
      max_new_tokens=60,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )

response = tokenizer.decode(output[0], skip_special_tokens=True)

if len(output) == 0:
    print("Warning: Model output is empty!")
else:
    print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


English: I feel sorry for her.
Indonesian: Aku merasa kasihan padanya.

English: I am always with you.
Indonesian: Saya akan selalu bersamamu.

English: We didn't understand much of what he said, only that he was suffering.
Indonesian: Kami tidak mengerti apa yang ia katakan, selain bahwa dia menderita.

English: Hey, good morning!
Indonesian: Hei, selamat pagi!

English: That's never happened.
Indonesian: Itu tidak pernah terjadi.

English: It's just, the way I asked you, I kind of left out the most important part.
Indonesian: Hanya saja, caraku menanyakannya padamu, aku agak merasa kelewatan bagian yang paling pentingnya.

translate English to Indonesian: 

English: Yeah, we've been teaching him, but really, he's been teaching us.
Indonesian: Ya, kita telah mengajarnya, tapi sebenarnya, dia telah mengajar kita.


In [9]:
# Test Data Examples
# english_sentences =
#     "Can I help you?"
#     "As you can see, I'm jet-lagged and trying to adjust to the time-zone change."
#     "That's what the policeman just said."
#     "Yeah, we've been teaching him, but really, he's been teaching us."
#     "Someone wants to kill him."
#     "Hey Four, that's pretty good right?"
#     "You should've told me, boy."
#     "Your bed is in the corner, over there."
#     "Hey, kid, you've done really good so far, so what we're gonna do right now is we're gonna call your daddy."

# indonesian_targets =
#     "ada yang bisa kubantu?"
#     "Seperti yang kamu lihat , aku alami jet - lag dan mencoba untuk menyesuaikan diri dengan perubahan zona waktu ."
#     "Begitulah kata polisinya tadi."
#     "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
#     "Seseorang ingin membunuhnya."
#     "Four, Tak begitu buruk, kan?"
#     "Kau harusnya katakan padaku, Nak."
#     "Ranjangmu di sudut itu."
#     "Hei, nak, sejauh ini kau telah melakukannya dengan baik sekali, jadi apa yang akan kita lakukan sekarang adalah kita akan menelpon ayahmu."

# Single reference translation
reference_sentence = "Ya, kami telah mengajari dia, tapi sungguh, ia telah mengajari kita."
candidate_sentence = "Ya, kita telah mengajar dia, tapi sebenarnya dia telah mengajar kita."

# Calculate BLEU score
bleu_score = calculate_bleu_single_reference(reference_sentence, candidate_sentence)
print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.1030


In [10]:
import pandas as pd

In [11]:
def extract_translation(output):
    """
    Extract the translation from the model's output after the second 'Indonesian:'.

    Parameters:
        output (str): The raw output from the model.

    Returns:
        str: The extracted translation.
    """
    # Split the output by 'Indonesian:'
    parts = output.split("Indonesian:")
    if len(parts) > 1:
        # Extract the part after the last occurrence of 'Indonesian:'
        translation = parts[-1].strip()  # Extract the text after the final 'Indonesian:'
    else:
        # If 'Indonesian:' is not found, return an empty string or handle as needed
        translation = ""

    return translation

In [12]:
print(extract_translation(response))

Ya, kita telah mengajarnya, tapi sebenarnya, dia telah mengajar kita.


In [13]:
def evaluate_model(model, tokenizer, test_data, output_path, english_sentences, indonesian_targets):
    results = []
    total_bleu_score = 0
    count = 0

    for example in test_data:
        # Extract input and target from the dataset
        english_sentence = example["translation"]["en"]
        reference_sentence = example["translation"]["id"]

        # Generate the prompt
        prompt = generate_few_shot_prompt(english_sentences, indonesian_targets, english_sentence)

        # Tokenize and generate prediction
        inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
        output = model.generate(
          input_ids=inputs,
          max_new_tokens=60,
          num_return_sequences=1,
          eos_token_id=tokenizer.eos_token_id,
        )
        raw_output = tokenizer.decode(output[0], skip_special_tokens=True)
        prediction_sentence = extract_translation(raw_output)

        # Calculate BLEU score
        bleu_score = calculate_bleu_single_reference(reference_sentence, prediction_sentence)
        total_bleu_score += bleu_score
        count += 1

        # Store results
        results.append({
            "Input (English)": english_sentence,
            "Reference (Indonesian)": reference_sentence,
            "Prediction (Indonesian)": prediction_sentence,
            "BLEU Score": bleu_score,
        })

    average_bleu_score = total_bleu_score / count if count > 0 else 0

    # Save results to Excel
    results_df = pd.DataFrame(results)
    results_df.to_excel(output_path, index=False)
    print(f"Results saved to {output_path}")
    print(f"Average BLEU Score: {average_bleu_score:.4f}")

    return results_df

In [ ]:
# Path
one_shot_path = "/content/drive/MyDrive/Colab Notebooks/phi-3_fine_tuned_one_shot_results.xlsx"

# Evaluate the model on the test data and save results
results_df_one_shot = evaluate_model(model, tokenizer, test_data, one_shot_path, english_sentences, indonesian_targets)

results_df_one_shot.head()

Results saved to /content/drive/MyDrive/Colab Notebooks/phi-3_fine_tuned_one_shot_results.xlsx
Average BLEU Score: 0.1432


,Input (English),Reference (Indonesian),Prediction (Indonesian),BLEU Score
0,He was on the run.,Dia sedang dalam pelarian.,Dia berlari.,0.069373
1,"How d'ya like that, Spidey?","Bagaimana sekarang, Spidey?","Apa kau suka itu, Spidey?",0.163481
2,"- Staff Sergeant, sir!","- Staf Sersan, Pak!","- Staff Sergeant, sir!",0.053728
3,I'll be with her.,Aku akan bersamanya.,Aku akan bertemu dengan dia.,0.095544
4,"Because of us, they will be ruined.","Karena kita, mereka akan hancur.","Karena kita, mereka akan rusak.",0.643459


In [14]:
few_shot_path = "/content/drive/MyDrive/Colab Notebooks/phi-3_fine_tuned_few_shot_results.xlsx"

# Training Data Examples
english_sentences = [
    "I feel sorry for her.",
    "I am always with you.",
    "We didn't understand much of what he said, only that he was suffering.",
    "Hey, good morning!",
    "That's never happened.",
    "It's just, the way I asked you, I kind of left out the most important part."
]

indonesian_targets = [
    "Aku merasa kasihan padanya.",
    "Saya akan selalu bersamamu.",
    "Kami tidak mengerti apa yang ia katakan, selain bahwa dia menderita.",
    "Hei, selamat pagi!",
    "Itu tidak pernah terjadi.",
    "Hanya saja, caraku menanyakannya padamu, aku agak merasa kelewatan bagian yang paling pentingnya."
]

results_df_few_shot = evaluate_model(model, tokenizer, test_data, few_shot_path, english_sentences, indonesian_targets)

results_df_few_shot.head()

Results saved to /content/drive/MyDrive/Colab Notebooks/phi-3_fine_tuned_few_shot_results.xlsx
Average BLEU Score: 0.1408


,Input (English),Reference (Indonesian),Prediction (Indonesian),BLEU Score
0,He was on the run.,Dia sedang dalam pelarian.,Dia berlari.,0.069373
1,"How d'ya like that, Spidey?","Bagaimana sekarang, Spidey?","Bagaimana kau suka dengan itu, Spidey?",0.147722
2,"- Staff Sergeant, sir!","- Staf Sersan, Pak!","- Staff Sergeant, sir!",0.053728
3,I'll be with her.,Aku akan bersamanya.,Aku akan bersamamu.,0.188030
4,"Because of us, they will be ruined.","Karena kita, mereka akan hancur.","Karena kita, mereka akan rusak.",0.643459
